In [83]:
import pandas as pd
import nltk

from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# Introdução
    Esse documento tem como objetivo analisar a expansão de consultas. Para isso serão respondidas algumas questões sobre o assunto. O documento anterior de notícias, disponível em: https://canvas.instructure.com/courses/1310917/files/63843198/download?verifier=wWhCU9ocjBa1uPBQFiNgCPIxgnShIFESev8TlF7p foi utilizado como nossa base de dados. 

# Funções auxiliares:
    Para responder as perguntas do final do documento foram criadas algumas funções auxiliares que estão descritas a seguir:

In [84]:
#Função para carregar o csv, preencher os espaços NaN e formatar o contéudo que ira ser utilizado
def getCSVFormattedAsDF():
    df = pd.read_csv(r'estadao_noticias_eleicao.csv')
    df = df.fillna('')
    df["docs"] = df["titulo"] + ' ' +  df["subTitulo"] + ' ' + df["conteudo"]
    return df

#Função para remover as pontuações dos documentos
def removePunctuationDocs(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_lists = docs.apply(lambda text: tokenizer.tokenize(text.lower()))
    return tokens_lists

#Função para remover stopwords. 
#stopwords: https://pt.wikipedia.org/wiki/Palavra_vazia
def removeStopWords(tokens_lists):
    stopword = stopwords.words('portuguese')
    filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword])
    return filtered_tokens

#Função para gerar os tokens de cada documento do dataframe inicial
def getTokens(docs):
    tokens = removePunctuationDocs(df["docs"])
    filtered_tokens = removeStopWords(tokens)
    return filtered_tokens

In [86]:
"""
    Função que retorna a matrix de co-ocorrência. 
    return matrix:
       [{"palavra1": [("palavrax", num_ocorrencias), ("palavray": num_ocorrencias), ...]
        "palavra2":  {(palavrak": num_ocorrencias), ("word3": palavraz) ... ]
"""
def co_occurrence_matrix(docs_tokens):
    docs_bi_grams = docs_tokens.apply(lambda tokens: list(bigrams(tokens)))
    collection_big_grams = [bi_gram for doc_big_grams in docs_bi_grams for bi_gram in doc_big_grams]
    bigram_freq = nltk.FreqDist(collection_big_grams).most_common(len(collection_big_grams))
    matrix = {}
    
    for bigram in bigram_freq:
        postWord = bigram[0][1]
        word = bigram[0][0]
        count = bigram[1]
        co_occurrence_info = (postWord, count)
        
        if word not in matrix:
            matrix[word] = []
        matrix[word].append(co_occurrence_info)

        
    return matrix

In [87]:
#Função que retorna as 3 maiores incidências de co-ocorrência com a busca em questão
def top3Consultas(word, matrix):
    if (word in matrix):
        co_occurrence_info = matrix[word]
        info = [(), (), ()]
        for index in range(len(co_occurrence_info)):
            info[index] = co_occurrence_info[index]
            if (index == 2):
                break
        return info

In [88]:
#inicia o indice invertido
def init_tokens(docs_tokens):
    dict_token = {}
    idx = -1
    for doc_token in docs_tokens:
        idx += 1
        for token in doc_token:
            if (token in dict_token):
                dict_token[token].add(idx) 
            else:
                dict_token[token] = set()
                dict_token[token].add(idx)
    return dict_token

In [89]:
# Retorna o conjunto de tokens da operação pesquisa. # Retorna 
def search(ipt, tokens):
    if (ipt.strip() == ""):
        return "Please, inform at least one token"
    
    ipt = ipt.split(" ")
    init = ipt[0].lower()
    size = len(ipt)
    
    if (size == 1) :
        if init not in tokens:
            return "This token is not present in any news"
        else:
            return tokens[init]
    else:
        result = tokens[init]
        command = ""
        for index in range(1,size):
            if (index % 2 == 0):
                compare = tokens[ipt[index].lower()]
    
                if  command == AND:
                    result = result & compare
                else:
                    result = result | compare              
            else:
                if (ipt[index] != "AND" and ipt[index] != "OR"):
                    return "Incorrect input. The tokens searched only can be separated by AND-OR"
                elif ipt[index] == "AND":
                    command = AND
                else:
                    command = OR
    
        return set(result)

# Variaveis auxiliares:
    A partir das funções auxiliares descritas na seção anterior, criamos algumas váriaveis auxiliares que nos ajudarão a deixar o processo mais rápido:

In [90]:
# dataFrame com os documentos lidos no csv
df = getCSVFormattedAsDF()
# Váriavel que armazena os tokens de palavras de cada notícia 
docs_tokens = getTokens(df["docs"])
# Váriavel que armazena a matrix de co-ocorrência
matrix_co_occurence = co_occurrence_matrix(docs_tokens)

#constantes necessários para o search
AND = 1
OR=  2



In [91]:
#tokenização necessário para o search
tokens = init_tokens(docs_tokens)

# Palavras escolhidas:
        Para a realização desta atividade seria necessário a escolha de três palavras inicias para poder fazer a busca. Escolhemos as palavras: petrobrás, lula e pt

# Perguntas
## Quais os termos retornados para a expansão de cada consulta?

In [92]:
top3Consultas("petrobrás", matrix_co_occurence)

[('paulo', 240), ('é', 90), ('graça', 51)]

In [93]:
top3Consultas("lula", matrix_co_occurence)

[('silva', 1028), ('dilma', 198), ('disse', 106)]

In [94]:
top3Consultas("pt", matrix_co_occurence)

[('psdb', 376), ('pmdb', 262), ('governo', 262)]

### Como visto, os resultados foram:

In [ ]:
[('paulo', 240), ('é', 90), ('graça', 51)]
[('silva', 1028), ('dilma', 198), ('disse', 106)]
[('psdb', 376), ('pmdb', 262), ('governo', 262)]

Ou seja, para a busca petrobrás, paulo foi a co-ocorrência mais encontrada. Para lula, temos silva. 
E por último, para pt, temos psdb.

## Você acha que esses termos são de fato relacionados com a consulta original? Justifique.



  <div style="text-align: justify;">
       <p> &emsp; De uma certa forma? Sim. Na primeira pesquisa temos petrobrás. Os termos encontrados foram:
    "Paulo", "Graça" e "é". Devido a base de dados, acredita-se que Paulo e Graça devam se referir, respectivamente, a Paulo Roberto Costa e Graça Foster, ex-diretores da petrobrás. A palavra "é" indica que a função removeStopWords deve ser melhorada.
       </p>
    <p>&emsp;
    Na segunda pesquisa("lula"), obtivemos como resultado: silva, dilma e disse. O que também mostra relação, já que silva faz parte do nome do ex-presidente e dilma está diretamente ligada com o nome de lula.
    </p>
    <p>&emsp;
    Na terceira pesquisa("pt") encontramos: "psdb", "pmdb", "governo", o que mostra uma relação, já que psdb e pmdb são partidos politicos e o pt governou o país por mais de 14 anos. 
    </p>
</div>
    
  

## Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que? A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

Ps: Responderei as duas em um texo só

In [ ]:
len(search("petrobrás", tokens))
len(search(["petrobrás", "paulo", "é", "graça"], tokens))
len(search("lula", tokens))
len(search(["lula", "silva", "dilma", "disse"], tokens))
len(search("pt", tokens))
len(search(["pt", "psdb", "pmdb", "governo"], tokens))

<table style="width:100%">
  <tr>
    <th>Termo</th>
    <th>Busca Expandida</th> 
    <th>Num Docs</th>
    <th>Num Docs Busca Expandida</th>
  </tr>
  <tr>
    <td>petrobrás</td>
    <td>petrobrás, paulo, é, graça</td> 
    <td>1043</td>
    <td>7200</td>
  </tr>
  <tr>
    <td>lula</td>
    <td>lula, silva, dilma, disse</td> 
    <td>1893</td>
    <td>6405</td>
  </tr>
  <tr>
    <td>pt</td>
    <td>pt, psdb, pmdb, governo</td> 
    <td>4018</td>
    <td>6509</td>
  </tr>
</table>

<div style="text-align:justify"> Acho que a captura da necessidade depende do contexto. Nota-se que a busca expansiva coleta mais documentos do que a original. Por um ponto de vista pode ser bom, já que sua busca vai retornar mais documentos que podem se relacionar com sua pesquisa, já que sua busca contém termos que se relacionam com o termo original. Ou seja, a pesquisa vai melhorar o <b>recall</b>. Contudo, se eu quero <b>precisão</b>, a consulta expandida pode não retornar os resultados mais precisos. Por exemplo, na busca do termo petrobrás, temos o termo "é" em nossa consulta expandida. O termo "é" é um termo comum na lingua portuguesa, que provavelmente deve ter aparecido em vários documentos que podem não ter uma relação muito direta com o termo original. O mesmo exemplo vale para o termo "disse" na busca por "lula". </div>